In [1]:
!pip install ultralytics

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 903.1/903.1 kB 14.1 MB/s eta 0:00:00a 0:00:01


In [2]:
from ultralytics import YOLO
import torch
import torch.nn as nn


# Define Triplet Attention
class TripletAttention(nn.Module):
    def __init__(self, channels, kernel_size=7):
        super(TripletAttention, self).__init__()
        self.conv1 = nn.Conv2d(channels, channels, kernel_size=(kernel_size, 1), padding=(kernel_size // 2, 0), bias=False)
        self.conv2 = nn.Conv2d(channels, channels, kernel_size=(1, kernel_size), padding=(0, kernel_size // 2), bias=False)
        self.conv3 = nn.Conv2d(channels, channels, kernel_size=kernel_size, padding=kernel_size // 2, bias=False)
        self.sigmoid = nn.Sigmoid()

    def forward(self, x):
        x1 = self.conv1(x.permute(0, 2, 3, 1)).permute(0, 3, 1, 2)  # Rotate and back
        x2 = self.conv2(x.permute(0, 3, 1, 2)).permute(0, 2, 3, 1)  # Rotate and back
        x3 = self.conv3(x)                                          # Original shape
        return self.sigmoid(x1 + x2 + x3)

Creating new Ultralytics Settings v0.0.6 file ✅ 
View Ultralytics Settings with 'yolo settings' or at '/root/.config/Ultralytics/settings.json'
Update Settings with 'yolo settings key=value', i.e. 'yolo settings runs_dir=path/to/dir'. For help see https://docs.ultralytics.com/quickstart/#ultralytics-settings.


In [3]:
from torchvision.ops import box_iou

def soft_nms(boxes, scores, iou_threshold=0.5, sigma=0.5, top_k=200):
    """
    Soft Non-Maximum Suppression (Soft-NMS).
    Args:
        boxes: (tensor) Bounding box coordinates (N, 4), where N is the number of boxes.
        scores: (tensor) Confidence scores for each bounding box (N,).
        iou_threshold: (float) IoU threshold for suppression.
        sigma: (float) Gaussian function parameter.
        top_k: (int) Number of top boxes to keep.
    Returns:
        kept_boxes: (tensor) The boxes that remain after Soft-NMS.
        kept_scores: (tensor) The scores of the kept boxes.
    """
    N = boxes.size(0)
    if N == 0:
        return boxes, scores

    # Initialize kept indices
    keep = []
    
    # Get the coordinates and scores sorted by the confidence score
    _, idxs = scores.sort(0, descending=True)

    while len(idxs) > 0:
        # Get the highest score box
        idx = idxs[0]
        keep.append(idx.item())
        
        if len(idxs) == 1:
            break
        
        # Get IoU between the highest score box and the rest
        iou = box_iou(boxes[idx].unsqueeze(0), boxes[idxs[1:]])  # (1, N-1)
        iou = iou.squeeze(0)

        # Apply Gaussian function for Soft-NMS
        weight = torch.exp(-(iou**2) / sigma)
        scores[idxs[1:]] = scores[idxs[1:]] * weight
        
        # Keep boxes with scores > threshold
        idxs = idxs[1:][scores[idxs[1:]] > 0.5]

    # Get the top-K results
    kept_boxes = boxes[keep]
    kept_scores = scores[keep]

    if len(kept_scores) > top_k:
        _, idxs = kept_scores.sort(0, descending=True)
        kept_boxes = kept_boxes[idxs[:top_k]]
        kept_scores = kept_scores[idxs[:top_k]]

    return kept_boxes, kept_scores


In [4]:
from ultralytics import YOLO
import torch.nn as nn

class custom_YOLO(YOLO):
    def __init__(self, model_path, nms_threshold=0.5, sigma=0.5, top_k=200):
        super().__init__(model_path)
        self.attention = TripletAttention(channels=512)  # Adjust channels based on backbone output size
        self.nms_threshold = nms_threshold
        self.sigma = sigma
        self.top_k = top_k

    def forward(self, x):
        # Run YOLO model to get predictions
        predictions = super().forward(x)

        # After obtaining the predictions, we perform Soft-NMS
        boxes, scores, labels = predictions[..., :4], predictions[..., 4], predictions[..., 5]

        # Apply Triplet Attention to the feature map (before the detection head)
        features = self.model.model[0:10](x)  # Forward pass through backbone layers (first 10 layers)
        features = self.attention(features)  # Apply Triplet Attention
        predictions = self.model.model[10:](features)  # Forward pass through neck and head

        # Run Soft-NMS
        boxes, scores = soft_nms(boxes, scores, iou_threshold=self.nms_threshold, sigma=self.sigma, top_k=self.top_k)
        
        # Reorganize output into required format (same format as YOLO output)
        predictions = torch.cat([boxes, scores.unsqueeze(1), labels.unsqueeze(1)], dim=1)
        return predictions


In [5]:
# Load YOLOv8 Model and Modify
model_path = "yolov8n.pt"  # YOLOv8 Nano model
custom_model = custom_YOLO(model_path)

100%|██████████| 6.25M/6.25M [00:00<00:00, 85.2MB/s]


In [6]:
from ultralytics import YOLO

# Load the YOLO model
train_images = "/kaggle/input/detect-classify/train"
val_images = "/kaggle/input/detect-classify/val"
num_classes = 2
classes = ['non_helmet', 'helmet']

import yaml

# Define paths and classes
data_config = {
    'train': train_images,
    'val': val_images,
    'nc': num_classes,
    'names': classes
}

# Save to a YAML file
with open('data.yaml', 'w') as file:
    yaml.dump(data_config, file)

In [7]:
!pip install -U ipywidgets

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 139.8/139.8 kB 3.1 MB/s eta 0:00:00a 0:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 214.4/214.4 kB 7.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.3/2.3 MB 45.8 MB/s eta 0:00:0000:01
  Attempting uninstall: widgetsnbextension
    Found existing installation: widgetsnbextension 3.6.9
    Uninstalling widgetsnbextension-3.6.9:
      Successfully uninstalled widgetsnbextension-3.6.9
  Attempting uninstall: jupyterlab-widgets
    Found existing installation: jupyterlab_widgets 3.0.11
    Uninstalling jupyterlab_widgets-3.0.11:
      Successfully uninstalled jupyterlab_widgets-3.0.11
  Attempting uninstall: ipywidgets
    Found existing installation: ipywidgets 7.7.1
    Uninstalling ipywidgets-7.7.1:
      Successfully uninstalled ipywidgets-7.7.1
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
big

In [8]:
custom_model.train(
    imgsz=640,
    data="data.yaml",  # Path to dataset YAML file
    epochs=50,                # Number of epochs
    batch=64,                 # Batch size
    name="yolov8_custom",  # Training run name
)

Ultralytics 8.3.54 🚀 Python-3.10.14 torch-2.4.0 CUDA:0 (Tesla T4, 15095MiB)
engine/trainer: task=detect, mode=train, model=yolov8n.pt, data=data.yaml, epochs=50, time=None, patience=100, batch=64, imgsz=640, save=True, save_period=-1, cache=False, device=None, workers=8, project=None, name=yolov8_custom, exist_ok=False, pretrained=True, optimizer=auto, verbose=True, seed=0, deterministic=True, single_cls=False, rect=False, cos_lr=False, close_mosaic=10, resume=False, amp=True, fraction=1.0, profile=False, freeze=None, multi_scale=False, overlap_mask=True, mask_ratio=4, dropout=0.0, val=True, split=val, save_json=False, save_hybrid=False, conf=None, iou=0.7, max_det=300, half=False, dnn=False, plots=True, source=None, vid_stride=1, stream_buffer=False, visualize=False, augment=False, agnostic_nms=False, classes=None, retina_masks=False, embed=None, show=False, save_frames=False, save_txt=False, save_conf=False, save_crop=False, show_labels=True, show_conf=True, show_boxes=True, line_wid

100%|██████████| 755k/755k [00:00<00:00, 17.4MB/s]


Overriding model.yaml nc=80 with nc=2

                   from  n    params  module                                       arguments                     
  0                  -1  1       464  ultralytics.nn.modules.conv.Conv             [3, 16, 3, 2]                 
  1                  -1  1      4672  ultralytics.nn.modules.conv.Conv             [16, 32, 3, 2]                
  2                  -1  1      7360  ultralytics.nn.modules.block.C2f             [32, 32, 1, True]             
  3                  -1  1     18560  ultralytics.nn.modules.conv.Conv             [32, 64, 3, 2]                
  4                  -1  2     49664  ultralytics.nn.modules.block.C2f             [64, 64, 2, True]             
  5                  -1  1     73984  ultralytics.nn.modules.conv.Conv             [64, 128, 3, 2]               
  6                  -1  2    197632  ultralytics.nn.modules.block.C2f             [128, 128, 2, True]           
  7                  -1  1    295424  ultralytics

100%|██████████| 5.35M/5.35M [00:00<00:00, 74.1MB/s]


AMP: checks passed ✅


...

                   all       2576       9080      0.787      0.759      0.802      0.687



50 epochs completed in 4.464 hours.
Optimizer stripped from runs/detect/yolov8_custom/weights/last.pt, 6.2MB
Optimizer stripped from runs/detect/yolov8_custom/weights/best.pt, 6.2MB

Validating runs/detect/yolov8_custom/weights/best.pt...
Ultralytics 8.3.54 🚀 Python-3.10.14 torch-2.4.0 CUDA:0 (Tesla T4, 15095MiB)
Model summary (fused): 168 layers, 3,006,038 parameters, 0 gradients, 8.1 GFLOPs


                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 21/21 [00:21<00:00,  1.04s/it]


                   all       2576       9080      0.791      0.782      0.828      0.703
            non_helmet       1581       3450      0.785      0.715      0.789      0.648
                helmet       2114       5630      0.798      0.848      0.868      0.758
Speed: 0.1ms preprocess, 1.2ms inference, 0.0ms loss, 0.9ms postprocess per image
Results saved to runs/detect/yolov8_custom


ultralytics.utils.metrics.DetMetrics object with attributes:

ap_class_index: array([0, 1])
box: ultralytics.utils.metrics.Metric object
confusion_matrix: <ultralytics.utils.metrics.ConfusionMatrix object at 0x78c7d3e96e00>
curves: ['Precision-Recall(B)', 'F1-Confidence(B)', 'Precision-Confidence(B)', 'Recall-Confidence(B)']
curves_results: [[array([          0,    0.001001,    0.002002,    0.003003,    0.004004,    0.005005,    0.006006,    0.007007,    0.008008,    0.009009,     0.01001,    0.011011,    0.012012,    0.013013,    0.014014,    0.015015,    0.016016,    0.017017,    0.018018,    0.019019,     0.02002,    0.021021,    0.022022,    0.023023,
          0.024024,    0.025025,    0.026026,    0.027027,    0.028028,    0.029029,     0.03003,    0.031031,    0.032032,    0.033033,    0.034034,    0.035035,    0.036036,    0.037037,    0.038038,    0.039039,     0.04004,    0.041041,    0.042042,    0.043043,    0.044044,    0.045045,    0.046046,    0.047047,
          0.04804